# 我是说明，请阅读

我们需要的**原始数据**字段：**日期,广告形式,产品,展现量,点击量,点击率,总加购数,转化率,ROI,总订单行,总订单金额,总费用。**

我们建立的**因变量：点击率/转化率，销售额/ROI，点击量/展现量，加购数/点击量，订单数/加购数**

我们的**自变量：广告类型的0-1变量，每日每渠道的投入金额（总费用）**

**目前的数据频次为每天，如果要改数据频次，因变量不能直接加和，因为他们都是比率**

京东数据应该可以直接生成下来，天猫数据需要改代码里面一些变量的名称之后才能相除，或者也可以将天猫数据的column改成京东的样子（第一行原始数据字段）

**京东**

京东所有广告类型：PUSH海投,购物触点,海投计划，京东展位，京东直投，京选店铺，经典海投，快车，首焦海投

分成五类：
展位（京东展位）、触点（购物触点）、海投（PUSH海投，海投计划，经典海投，首焦海投）、快车（快车，京选店铺）、其他（京东直投）

**天猫**

天猫所有广告类型都是分成sheet放的，有四种：超级推荐，直通车，钻展，品销宝

或许需要手动加变量，格式：在每一页加上四列类型变量（是否为超级推荐，是否为直通车，是否为钻展，是否为品销宝；是的话用1，不是用0）

天猫每个sheet的变量不一样，需要分别处理：

超级推荐：时间，展现，点击，点击率，添加购物车量，点击转化率，投资回报率，成交订单量，成交订单金额，消耗

直通车：日期，展现量，点击量，点击率，总购物车数，点击转化率，投入产出比，总成交笔数，总成交金额(分)，花费(分)

钻展：日期，有效展现，有效点击数，点击率，添加购物车量，点击转化率，投资回报率，成交订单量，成交订单金额，消耗

品销宝：日期，展现量，点击量，点击率，宝贝加购数，转化率，回报率，成交笔数，成交金额，消耗

每个sheet都要加上这四个变量：是否为超级推荐，是否为直通车，是否为钻展，是否为品销宝，让该类别全部为1，其他全部为0


In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('京东二级子类每日数据求和汇总.csv',encoding='gb18030')
df2 = df.loc[:,['日期','广告二级类型','展现量','点击量','点击率','总加购数','转化率','ROI','总订单行','总订单金额','总费用']]
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528 entries, 0 to 1527
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   日期      1528 non-null   object 
 1   广告二级类型  1528 non-null   object 
 2   展现量     1528 non-null   int64  
 3   点击量     1528 non-null   int64  
 4   点击率     1528 non-null   float64
 5   总加购数    1528 non-null   int64  
 6   转化率     1528 non-null   object 
 7   ROI     1528 non-null   object 
 8   总订单行    1528 non-null   int64  
 9   总订单金额   1528 non-null   float64
 10  总费用     1528 non-null   float64
dtypes: float64(3), int64(4), object(4)
memory usage: 131.4+ KB
None


In [2]:
df2 = df2.replace('#DIV/0!', np.NaN)
df2.loc[:,['点击率']] = pd.DataFrame(df2.loc[:,['点击率']],dtype = float)
df2.loc[:,['ROI']] = pd.DataFrame(df2.loc[:,['ROI']],dtype = float)
df2.loc[:,['转化率']] = pd.DataFrame(df2.loc[:,['转化率']],dtype = float)
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528 entries, 0 to 1527
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   日期      1528 non-null   object 
 1   广告二级类型  1528 non-null   object 
 2   展现量     1528 non-null   int64  
 3   点击量     1528 non-null   int64  
 4   点击率     1528 non-null   float64
 5   总加购数    1528 non-null   int64  
 6   转化率     1466 non-null   float64
 7   ROI     1471 non-null   float64
 8   总订单行    1528 non-null   int64  
 9   总订单金额   1528 non-null   float64
 10  总费用     1528 non-null   float64
dtypes: float64(5), int64(4), object(2)
memory usage: 131.4+ KB
None


效果衡量方式：

1.利用ROI

i.点击率/转化率

ii.销售额/ROI

2.利用各层转换率：点击量/展现量，加购数/点击量，订单数/加购数

i.PCA

ii.在三个中寻找最小的一个

In [3]:
df2

日期 广告二级类型     展现量   点击量   点击率  总加购数    转化率    ROI  总订单行    总订单金额  \
0     2019/5/11  展位类广告   25983   104  0.40     0   0.00   0.00     0      0.0   
1     2019/5/12  展位类广告  141307   697  0.49     1   0.00   0.00     0      0.0   
2     2019/5/13  展位类广告  248611  1054  0.42     7   0.00   0.00     0      0.0   
3     2019/5/14  展位类广告  232516   735  0.32     4   0.00   0.00     0      0.0   
4     2019/5/15  展位类广告  138092   457  0.33     3   0.00   0.00     0      0.0   
...         ...    ...     ...   ...   ...   ...    ...    ...   ...      ...   
1523  2021/3/27  海投类广告   17485   432  2.47   254  12.96  64.18    56  71638.0   
1524  2021/3/28  海投类广告   17488   426  2.44   180   6.34  27.90    27  27853.0   
1525  2021/3/29  海投类广告   45830   892  1.95   365  10.09  23.39    90  80822.0   
1526  2021/3/30  海投类广告   29439   727  2.47   290   9.35  34.58    68  75384.0   
1527  2021/3/31  海投类广告   26131   725  2.77   286   8.41  38.62    61  70218.0   

          总费用  
0      138.05  
1      857.95  
2     1841.42  
3     1739.28  
4      989.84  
...       ...  
1523  1116.17  
1524   998.33  
1525  3455.93  
1526  2180.07  
1527  1818.12  

[1528 rows x 11 columns]

In [4]:
size=len(df2)

In [5]:
#京东所有展示类广告可再细分为三种类型：展位类、触点类、海投类
#dummy1=展位类广告（京东展位）
#dummy2=触点类广告（购物触点）
#dummy3=海投类广告（PUSH海投、海投计划、海投新品、经典海投、首焦海投）

dummy1=[]
d1=['展位类广告']
dummy2=[]
d2=['触点类广告']
dummy3=[]
d3=['海投类广告']

for i in range(size):
    if df2['广告二级类型'][i] in d1:
        dummy1.append(1)
        dummy2.append(0)
        dummy3.append(0)
     
    elif df2['广告二级类型'][i] in d2:
        dummy1.append(0)
        dummy2.append(1)
        dummy3.append(0)

    elif df2['广告二级类型'][i] in d3:
        dummy1.append(0)
        dummy2.append(0)
        dummy3.append(1)

dummy_1=pd.DataFrame(dummy1)
dummy_2=pd.DataFrame(dummy2)
dummy_3=pd.DataFrame(dummy3)

df2['是否为展位类广告']=dummy_1
df2['是否为触点类广告']=dummy_2
df2['是否为海投类广告']=dummy_3

df2

日期 广告二级类型     展现量   点击量   点击率  总加购数    转化率    ROI  总订单行    总订单金额  \
0     2019/5/11  展位类广告   25983   104  0.40     0   0.00   0.00     0      0.0   
1     2019/5/12  展位类广告  141307   697  0.49     1   0.00   0.00     0      0.0   
2     2019/5/13  展位类广告  248611  1054  0.42     7   0.00   0.00     0      0.0   
3     2019/5/14  展位类广告  232516   735  0.32     4   0.00   0.00     0      0.0   
4     2019/5/15  展位类广告  138092   457  0.33     3   0.00   0.00     0      0.0   
...         ...    ...     ...   ...   ...   ...    ...    ...   ...      ...   
1523  2021/3/27  海投类广告   17485   432  2.47   254  12.96  64.18    56  71638.0   
1524  2021/3/28  海投类广告   17488   426  2.44   180   6.34  27.90    27  27853.0   
1525  2021/3/29  海投类广告   45830   892  1.95   365  10.09  23.39    90  80822.0   
1526  2021/3/30  海投类广告   29439   727  2.47   290   9.35  34.58    68  75384.0   
1527  2021/3/31  海投类广告   26131   725  2.77   286   8.41  38.62    61  70218.0   

          总费用  是否为展位类广告  是否为触点类广告  是否为海投类广告  
0      138.05         1         0         0  
1      857.95         1         0         0  
2     1841.42         1         0         0  
3     1739.28         1         0         0  
4      989.84         1         0         0  
...       ...       ...       ...       ...  
1523  1116.17         0         0         1  
1524   998.33         0         0         1  
1525  3455.93         0         0         1  
1526  2180.07         0         0         1  
1527  1818.12         0         0         1  

[1528 rows x 14 columns]

In [6]:
#index1=点击率/转化率
index1=[]
for i in range(size):
    if df2['转化率'][i] !=0:
        index=df2['点击率'][i]/df2['转化率'][i]
    else:
        index=0
    index1.append(index)
index_1=pd.DataFrame(index1)
df2['点击率/转化率']=index_1
df2

日期 广告二级类型     展现量   点击量   点击率  总加购数    转化率    ROI  总订单行    总订单金额  \
0     2019/5/11  展位类广告   25983   104  0.40     0   0.00   0.00     0      0.0   
1     2019/5/12  展位类广告  141307   697  0.49     1   0.00   0.00     0      0.0   
2     2019/5/13  展位类广告  248611  1054  0.42     7   0.00   0.00     0      0.0   
3     2019/5/14  展位类广告  232516   735  0.32     4   0.00   0.00     0      0.0   
4     2019/5/15  展位类广告  138092   457  0.33     3   0.00   0.00     0      0.0   
...         ...    ...     ...   ...   ...   ...    ...    ...   ...      ...   
1523  2021/3/27  海投类广告   17485   432  2.47   254  12.96  64.18    56  71638.0   
1524  2021/3/28  海投类广告   17488   426  2.44   180   6.34  27.90    27  27853.0   
1525  2021/3/29  海投类广告   45830   892  1.95   365  10.09  23.39    90  80822.0   
1526  2021/3/30  海投类广告   29439   727  2.47   290   9.35  34.58    68  75384.0   
1527  2021/3/31  海投类广告   26131   725  2.77   286   8.41  38.62    61  70218.0   

          总费用  是否为展位类广告  是否为触点类广告  是否为海投类广告   点击率/转化率  
0      138.05         1         0         0  0.000000  
1      857.95         1         0         0  0.000000  
2     1841.42         1         0         0  0.000000  
3     1739.28         1         0         0  0.000000  
4      989.84         1         0         0  0.000000  
...       ...       ...       ...       ...       ...  
1523  1116.17         0         0         1  0.190586  
1524   998.33         0         0         1  0.384858  
1525  3455.93         0         0         1  0.193261  
1526  2180.07         0         0         1  0.264171  
1527  1818.12         0         0         1  0.329370  

[1528 rows x 15 columns]

In [7]:
#index2=销售额/ROI
index2=[]
for i in range(size):
    if df2['ROI'][i] !=0:
        index=df2['总订单金额'][i]/df2['ROI'][i]
    else:
        index=0
    index2.append(index)
index_2=pd.DataFrame(index2)
df2['销售额/ROI']=index_2

In [8]:
df2

日期 广告二级类型     展现量   点击量   点击率  总加购数    转化率    ROI  总订单行    总订单金额  \
0     2019/5/11  展位类广告   25983   104  0.40     0   0.00   0.00     0      0.0   
1     2019/5/12  展位类广告  141307   697  0.49     1   0.00   0.00     0      0.0   
2     2019/5/13  展位类广告  248611  1054  0.42     7   0.00   0.00     0      0.0   
3     2019/5/14  展位类广告  232516   735  0.32     4   0.00   0.00     0      0.0   
4     2019/5/15  展位类广告  138092   457  0.33     3   0.00   0.00     0      0.0   
...         ...    ...     ...   ...   ...   ...    ...    ...   ...      ...   
1523  2021/3/27  海投类广告   17485   432  2.47   254  12.96  64.18    56  71638.0   
1524  2021/3/28  海投类广告   17488   426  2.44   180   6.34  27.90    27  27853.0   
1525  2021/3/29  海投类广告   45830   892  1.95   365  10.09  23.39    90  80822.0   
1526  2021/3/30  海投类广告   29439   727  2.47   290   9.35  34.58    68  75384.0   
1527  2021/3/31  海投类广告   26131   725  2.77   286   8.41  38.62    61  70218.0   

          总费用  是否为展位类广告  是否为触点类广告  是否为海投类广告   点击率/转化率      销售额/ROI  
0      138.05         1         0         0  0.000000     0.000000  
1      857.95         1         0         0  0.000000     0.000000  
2     1841.42         1         0         0  0.000000     0.000000  
3     1739.28         1         0         0  0.000000     0.000000  
4      989.84         1         0         0  0.000000     0.000000  
...       ...       ...       ...       ...       ...          ...  
1523  1116.17         0         0         1  0.190586  1116.204425  
1524   998.33         0         0         1  0.384858   998.315412  
1525  3455.93         0         0         1  0.193261  3455.408294  
1526  2180.07         0         0         1  0.264171  2179.988433  
1527  1818.12         0         0         1  0.329370  1818.177110  

[1528 rows x 16 columns]

In [9]:
#index3=点击量/展现量
index3=[]
for i in range(size):
    if df2['展现量'][i] !=0:
        index=df2['点击量'][i]/df2['展现量'][i]
    else:
        index=0
    index3.append(index)
index_3=pd.DataFrame(index3)
df2['点击量/展现量']=index_3
df2

日期 广告二级类型     展现量   点击量   点击率  总加购数    转化率    ROI  总订单行    总订单金额  \
0     2019/5/11  展位类广告   25983   104  0.40     0   0.00   0.00     0      0.0   
1     2019/5/12  展位类广告  141307   697  0.49     1   0.00   0.00     0      0.0   
2     2019/5/13  展位类广告  248611  1054  0.42     7   0.00   0.00     0      0.0   
3     2019/5/14  展位类广告  232516   735  0.32     4   0.00   0.00     0      0.0   
4     2019/5/15  展位类广告  138092   457  0.33     3   0.00   0.00     0      0.0   
...         ...    ...     ...   ...   ...   ...    ...    ...   ...      ...   
1523  2021/3/27  海投类广告   17485   432  2.47   254  12.96  64.18    56  71638.0   
1524  2021/3/28  海投类广告   17488   426  2.44   180   6.34  27.90    27  27853.0   
1525  2021/3/29  海投类广告   45830   892  1.95   365  10.09  23.39    90  80822.0   
1526  2021/3/30  海投类广告   29439   727  2.47   290   9.35  34.58    68  75384.0   
1527  2021/3/31  海投类广告   26131   725  2.77   286   8.41  38.62    61  70218.0   

          总费用  是否为展位类广告  是否为触点类广告  是否为海投类广告   点击率/转化率      销售额/ROI   点击量/展现量  
0      138.05         1         0         0  0.000000     0.000000  0.004003  
1      857.95         1         0         0  0.000000     0.000000  0.004933  
2     1841.42         1         0         0  0.000000     0.000000  0.004240  
3     1739.28         1         0         0  0.000000     0.000000  0.003161  
4      989.84         1         0         0  0.000000     0.000000  0.003309  
...       ...       ...       ...       ...       ...          ...       ...  
1523  1116.17         0         0         1  0.190586  1116.204425  0.024707  
1524   998.33         0         0         1  0.384858   998.315412  0.024360  
1525  3455.93         0         0         1  0.193261  3455.408294  0.019463  
1526  2180.07         0         0         1  0.264171  2179.988433  0.024695  
1527  1818.12         0         0         1  0.329370  1818.177110  0.027745  

[1528 rows x 17 columns]

In [10]:
#index4=加购数/点击量
index4=[]
for i in range(size):
    if df2['点击量'][i] !=0:
        index=df2['总加购数'][i]/df2['点击量'][i]
    else:
        index=0
    index4.append(index)
index_4=pd.DataFrame(index4)
df2['加购数/点击量']=index_4
df2

日期 广告二级类型     展现量   点击量   点击率  总加购数    转化率    ROI  总订单行    总订单金额  \
0     2019/5/11  展位类广告   25983   104  0.40     0   0.00   0.00     0      0.0   
1     2019/5/12  展位类广告  141307   697  0.49     1   0.00   0.00     0      0.0   
2     2019/5/13  展位类广告  248611  1054  0.42     7   0.00   0.00     0      0.0   
3     2019/5/14  展位类广告  232516   735  0.32     4   0.00   0.00     0      0.0   
4     2019/5/15  展位类广告  138092   457  0.33     3   0.00   0.00     0      0.0   
...         ...    ...     ...   ...   ...   ...    ...    ...   ...      ...   
1523  2021/3/27  海投类广告   17485   432  2.47   254  12.96  64.18    56  71638.0   
1524  2021/3/28  海投类广告   17488   426  2.44   180   6.34  27.90    27  27853.0   
1525  2021/3/29  海投类广告   45830   892  1.95   365  10.09  23.39    90  80822.0   
1526  2021/3/30  海投类广告   29439   727  2.47   290   9.35  34.58    68  75384.0   
1527  2021/3/31  海投类广告   26131   725  2.77   286   8.41  38.62    61  70218.0   

          总费用  是否为展位类广告  是否为触点类广告  是否为海投类广告   点击率/转化率      销售额/ROI   点击量/展现量  \
0      138.05         1         0         0  0.000000     0.000000  0.004003   
1      857.95         1         0         0  0.000000     0.000000  0.004933   
2     1841.42         1         0         0  0.000000     0.000000  0.004240   
3     1739.28         1         0         0  0.000000     0.000000  0.003161   
4      989.84         1         0         0  0.000000     0.000000  0.003309   
...       ...       ...       ...       ...       ...          ...       ...   
1523  1116.17         0         0         1  0.190586  1116.204425  0.024707   
1524   998.33         0         0         1  0.384858   998.315412  0.024360   
1525  3455.93         0         0         1  0.193261  3455.408294  0.019463   
1526  2180.07         0         0         1  0.264171  2179.988433  0.024695   
1527  1818.12         0         0         1  0.329370  1818.177110  0.027745   

       加购数/点击量  
0     0.000000  
1     0.001435  
2     0.006641  
3     0.005442  
4     0.006565  
...        ...  
1523  0.587963  
1524  0.422535  
1525  0.409193  
1526  0.398900  
1527  0.394483  

[1528 rows x 18 columns]

In [11]:
#index5=订单数/加购数
index5=[]
for i in range(size):
    if df2['总订单行'][i] !=0:
        index=df2['总加购数'][i]/df2['总订单行'][i]
    else:
        index=0
    index5.append(index)
index_5=pd.DataFrame(index5)
df2['订单数/加购数']=index_5
df2

日期 广告二级类型     展现量   点击量   点击率  总加购数    转化率    ROI  总订单行    总订单金额  \
0     2019/5/11  展位类广告   25983   104  0.40     0   0.00   0.00     0      0.0   
1     2019/5/12  展位类广告  141307   697  0.49     1   0.00   0.00     0      0.0   
2     2019/5/13  展位类广告  248611  1054  0.42     7   0.00   0.00     0      0.0   
3     2019/5/14  展位类广告  232516   735  0.32     4   0.00   0.00     0      0.0   
4     2019/5/15  展位类广告  138092   457  0.33     3   0.00   0.00     0      0.0   
...         ...    ...     ...   ...   ...   ...    ...    ...   ...      ...   
1523  2021/3/27  海投类广告   17485   432  2.47   254  12.96  64.18    56  71638.0   
1524  2021/3/28  海投类广告   17488   426  2.44   180   6.34  27.90    27  27853.0   
1525  2021/3/29  海投类广告   45830   892  1.95   365  10.09  23.39    90  80822.0   
1526  2021/3/30  海投类广告   29439   727  2.47   290   9.35  34.58    68  75384.0   
1527  2021/3/31  海投类广告   26131   725  2.77   286   8.41  38.62    61  70218.0   

          总费用  是否为展位类广告  是否为触点类广告  是否为海投类广告   点击率/转化率      销售额/ROI   点击量/展现量  \
0      138.05         1         0         0  0.000000     0.000000  0.004003   
1      857.95         1         0         0  0.000000     0.000000  0.004933   
2     1841.42         1         0         0  0.000000     0.000000  0.004240   
3     1739.28         1         0         0  0.000000     0.000000  0.003161   
4      989.84         1         0         0  0.000000     0.000000  0.003309   
...       ...       ...       ...       ...       ...          ...       ...   
1523  1116.17         0         0         1  0.190586  1116.204425  0.024707   
1524   998.33         0         0         1  0.384858   998.315412  0.024360   
1525  3455.93         0         0         1  0.193261  3455.408294  0.019463   
1526  2180.07         0         0         1  0.264171  2179.988433  0.024695   
1527  1818.12         0         0         1  0.329370  1818.177110  0.027745   

       加购数/点击量   订单数/加购数  
0     0.000000  0.000000  
1     0.001435  0.000000  
2     0.006641  0.000000  
3     0.005442  0.000000  
4     0.006565  0.000000  
...        ...       ...  
1523  0.587963  4.535714  
1524  0.422535  6.666667  
1525  0.409193  4.055556  
1526  0.398900  4.264706  
1527  0.394483  4.688525  

[1528 rows x 19 columns]

In [35]:
#描述性统计

In [12]:
dependent = df2.loc[:,['点击率/转化率','销售额/ROI','点击量/展现量','加购数/点击量','订单数/加购数']]
independent = df2.loc[:,['是否为展位类广告','是否为触点类广告','是否为海投类广告','总费用']]

In [13]:
dependent.describe()

点击率/转化率       销售额/ROI      点击量/展现量      加购数/点击量      订单数/加购数
count  1466.000000   1471.000000  1528.000000  1528.000000  1528.000000
mean      0.892173   2223.171248     0.011096     0.182716    12.583150
std       2.029064   3140.212882     0.016007     0.227908    55.005042
min       0.000000      0.000000     0.000000     0.000000     0.000000
25%       0.186080    448.676616     0.007853     0.074396     2.825820
50%       0.425006   1314.473684     0.010262     0.137995     6.000000
75%       0.957761   2721.860566     0.013223     0.237028    11.212500
max      54.500000  32795.642785     0.600000     4.000000  1844.000000

In [14]:
independent.describe()

是否为展位类广告     是否为触点类广告     是否为海投类广告           总费用
count  1528.000000  1528.000000  1528.000000   1528.000000
mean      0.151178     0.410340     0.438482   2313.274490
std       0.358340     0.492056     0.496364   3059.539239
min       0.000000     0.000000     0.000000      0.000000
25%       0.000000     0.000000     0.000000    718.527500
50%       0.000000     0.000000     0.000000   1367.355000
75%       0.000000     1.000000     1.000000   2697.117500
max       1.000000     1.000000     1.000000  32798.870000

In [15]:
#生成输出文档
import csv
outfile=open('after.csv','w',newline='')
to_write=csv.writer(outfile)
title=['日期','广告二级类型','展现量','点击量', '点击率','总加购数','转化率','ROI','总订单行','总订单金额','总费用',
       '是否为展位类广告','是否为触点类广告','是否为海投类广告','点击率/转化率','销售额/ROI','点击量/展现量','加购数/点击量','订单数/加购数']
to_write.writerow(title)
for i in range(size):
    row = df2.iloc[i].values.tolist()
    to_write.writerow(row)
outfile.close()

**以下为PCA代码，不需要，但是保留着，万一有用呢**

In [14]:
#df3=df2.iloc[:,13:]
#df3

点击量/展现量   加购数/点击量  订单数/加购数
0      0.004479  0.000000      0.0
1      0.006029  0.125000      0.0
2      0.010406  0.000000      0.0
3      0.012303  0.000000      0.0
4      0.007307  0.071429      0.0
...         ...       ...      ...
33044  0.333333  0.000000      0.0
33045  0.008434  0.016260     12.0
33046  0.000000  0.000000      0.0
33047  0.027506  0.001350      0.0
33048  0.000000  0.000000      0.0

[33049 rows x 3 columns]

In [15]:
#PCA
#df3_T = df3.T
#df3_T

0         1         2         3         4         5         6      \
点击量/展现量  0.004479  0.006029  0.010406  0.012303  0.007307  0.004558  0.008522   
加购数/点击量  0.000000  0.125000  0.000000  0.000000  0.071429  0.800000  0.050000   
订单数/加购数  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  1.000000   

            7        8      9      ...     33039     33040     33041  33042  \
点击量/展现量  0.011516  0.00202    0.0  ...  0.032895  0.011228  0.009384    0.0   
加购数/点击量  0.083333  4.00000    0.0  ...  0.006192  0.000000  0.002225    0.0   
订单数/加购数  0.000000  0.00000    0.0  ...  0.000000  0.000000  0.000000    0.0   

           33043     33044      33045  33046     33047  33048  
点击量/展现量  0.02337  0.333333   0.008434    0.0  0.027506    0.0  
加购数/点击量  0.00000  0.000000   0.016260    0.0  0.001350    0.0  
订单数/加购数  0.00000  0.000000  12.000000    0.0  0.000000    0.0  

[3 rows x 33049 columns]

In [16]:
#from sklearn import decomposition
#pca = decomposition.PCA(n_components=1)
#pca.fit(df3_T)
#print(pca.transform(df3_T))
#print(pca.explained_variance_ratio_)

[[-176.39928398]
 [-170.76988765]
 [ 347.16917162]]
[0.96580412]


In [17]:
#index6=PCA结果
#index6=[]
#for i in range(size):
#    pca=df3['点击量/展现量'][i]*(-176.4)+df3['加购数/点击量'][i]*(-170.8)+df3['订单数/加购数'][i]*(347.2)
#    index6.append(pca)
#index_6=pd.DataFrame(index6)
#df2['pca']=index_6
#df2

日期  广告形式     产品      展现量    点击量   点击率  总加购数  转化率   ROI  总订单行  \
0      2019/5/30  购物触点  SR501    893.0    4.0  0.45   0.0  0.0  0.00   0.0   
1      2019/5/30  购物触点  SR501   1327.0    8.0  0.60   1.0  0.0  0.00   0.0   
2      2019/5/29  购物触点  SR501    961.0   10.0  1.04   0.0  0.0  0.00   0.0   
3      2019/5/29  购物触点  SR501   1463.0   18.0  1.23   0.0  0.0  0.00   0.0   
4      2019/5/28  购物触点  SR501   1916.0   14.0  0.73   1.0  0.0  0.00   0.0   
...          ...   ...    ...      ...    ...   ...   ...  ...   ...   ...   
33044  2019/12/2  京东直投   京东直投      6.0    2.0  0.33   0.0  0.0  0.00   0.0   
33045  2019/12/1  京东直投   京东直投  87498.0  738.0  0.01  12.0  0.0  0.28   1.0   
33046  2019/12/2  京东直投   京东直投     34.0    0.0  0.00   0.0  0.0  0.00   0.0   
33047  2019/12/1  京东直投   京东直投  26940.0  741.0  0.03   1.0  0.0  0.00   0.0   
33048  2019/12/2  京东直投   京东直投      8.0    0.0  0.00   0.0  0.0  0.00   0.0   

       总订单金额  点击率/转化率     销售额/ROI   点击量/展现量   加购数/点击量  订单数/加购数          pca  
0        0.0      0.0    0.000000  0.004479  0.000000      0.0    -0.790146  
1        0.0      0.0    0.000000  0.006029  0.125000      0.0   -22.413451  
2        0.0      0.0    0.000000  0.010406  0.000000      0.0    -1.835588  
3        0.0      0.0    0.000000  0.012303  0.000000      0.0    -2.170335  
4        0.0      0.0    0.000000  0.007307  0.071429      0.0   -13.488935  
...      ...      ...         ...       ...       ...      ...          ...  
33044    0.0      0.0    0.000000  0.333333  0.000000      0.0   -58.800000  
33045   79.0      0.0  282.142857  0.008434  0.016260     12.0  4162.134922  
33046    0.0      0.0    0.000000  0.000000  0.000000      0.0     0.000000  
33047    0.0      0.0    0.000000  0.027506  0.001350      0.0    -5.082482  
33048    0.0      0.0    0.000000  0.000000  0.000000      0.0     0.000000  

[33049 rows x 17 columns]

In [18]:
#df3.describe()

点击量/展现量       加购数/点击量       订单数/加购数
count  33049.000000  33049.000000  33049.000000
mean       0.024409      0.104120      0.633639
std        0.083388      0.625749      2.840181
min        0.000000      0.000000      0.000000
25%        0.000000      0.000000      0.000000
50%        0.000000      0.000000      0.000000
75%        0.013550      0.000000      0.000000
max        4.000000     59.000000    110.000000

In [ ]:
#选取 点击量/展现量